In [1]:
import numpy as np
import os
import librosa
import keras
import pandas as pd
import time

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

2023-05-30 01:27:00.079770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [5]:
path = 'BirdCLEF/birdnet_embeddings'
species = [f.split('.')[0] for f in os.listdir(path)][:10]
species

['bswdov1-27',
 'bawhor2-47',
 'categr-166',
 'bltbar1-7',
 'combul2-200',
 'carcha1-153',
 'brctch1-62',
 'barswa-500',
 'afpwag1-81',
 'afrthr1-45']

In [11]:
embeddings = [np.load('%s/%s.npy' % (path, s)) for s in species]

In [12]:
X = np.concatenate(embeddings)
y = np.concatenate([np.full(len(embeddings[i]), species[i]) for i in range(10)])

In [124]:
X.shape

(12546, 1024)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    shuffle=True,
                                                    test_size = 0.15, 
                                                    random_state = 431)

In [14]:
X_train.shape

(10664, 1024)

In [15]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
reg = LogisticRegression(max_iter=1000).fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(X_test)

In [17]:
y_test_pred = reg.predict(X_test_scaled)

In [18]:
correct = np.count_nonzero(y_test == y_test_pred)
all = len(y_test)
correct, all, correct/all

(1790, 1882, 0.9511158342189161)

In [19]:
y_train_pred = reg.predict(X_train_scaled)
correct = np.count_nonzero(y_train == y_train_pred)
all = len(y_train)
correct, all, correct/all

(10664, 10664, 1.0)

### Trying out some classifiers with and without PCA preprocessing

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

from matplotlib.pyplot import cm

In [25]:
## Make the kfold object here
kfold = KFold(n_splits = 5,
              shuffle=True,
              random_state= 423)

##### KNN without PCA (since it liked the higher dimension anyways)

In [26]:
ks = range(1, 51)
n_splits = 5

st = time.time()

## Makes a 3-D array of to record the accuracies in
knn_accs = np.zeros((n_splits, len(ks)))
knn_accs2 = np.zeros((n_splits, len(ks)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for neighbors in ks:
        dimst = time.time()
        knn = KNeighborsClassifier(neighbors)

        ## Fit on the tt data
        knn.fit(tt, y_tt)

        ## Get the holdout prediction
        pred = knn.predict(ho)

        knn_accs[i,j] = accuracy_score(y_ho, pred)
        knn_accs2[i,j] = accuracy_score(y_tt, knn.predict(tt))
        
        j = j + 1
        dimtime = time.time() - dimst
        print("Num Neighbors = %s" % neighbors)
        print(dimtime)
    i = i + 1
et = time.time() - st
print(et)

CV Split 0
Num Neighbors = 1
1.1198315620422363
Num Neighbors = 2
1.1278071403503418
Num Neighbors = 3
1.0587105751037598
Num Neighbors = 4
1.0743234157562256
Num Neighbors = 5
1.0496716499328613
Num Neighbors = 6
1.1017520427703857
Num Neighbors = 7
1.0389695167541504
Num Neighbors = 8
1.044992208480835
Num Neighbors = 9
1.0500023365020752
Num Neighbors = 10
1.0370731353759766
Num Neighbors = 11
1.0726933479309082
Num Neighbors = 12
1.0862066745758057
Num Neighbors = 13
1.0373139381408691
Num Neighbors = 14
1.0912432670593262
Num Neighbors = 15
1.1823914051055908
Num Neighbors = 16
1.1443371772766113
Num Neighbors = 17
1.106294870376587
Num Neighbors = 18
1.045180320739746
Num Neighbors = 19
1.0720529556274414
Num Neighbors = 20
1.0451509952545166
Num Neighbors = 21
1.0570313930511475
Num Neighbors = 22
1.0383589267730713
Num Neighbors = 23
1.087674856185913
Num Neighbors = 24
1.1517024040222168
Num Neighbors = 25
1.122746229171753
Num Neighbors = 26
1.0788395404815674
Num Neighbors =

In [62]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.argmax(np.mean(knn_accs, axis=0))

print(max_index)

print("The k with the highest AVG CV Accuracy was",
         "k =", ks[max_index])
print("The highest AVG CV Accuracy was", np.max(np.mean(knn_accs, axis=0)))

print("The corresponding AVG CV Accuracy was", np.mean(knn_accs2[:,max_index]))

0
The k with the highest AVG CV Accuracy was k = 1
The highest AVG CV Accuracy was 0.9635216806565989
The corresponding AVG CV Accuracy was 1.0


##### KNN with PCA

In [28]:
ks = range(1, 11)
comps = range(2,20)
n_splits = 5
## Makes a 3-D array of to record the accuracies in
pca_accs = np.zeros((n_splits, len(comps), len(ks)))
pca_accs2 = np.zeros((n_splits, len(comps), len(ks)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for n_comps in comps:
        ## Make the PCA pipeline here
        pca_pipe = Pipeline([('scale', StandardScaler()),
                            ('pca', PCA(n_comps))])
        
        ## Fit and then get the PCA transformed tt data here
        pca_tt = pca_pipe.fit_transform(tt)
        
        ## Get the transformed holdout data here
        pca_ho = pca_pipe.transform(ho)
        
        k = 0
        for neighbors in ks:
            knn = KNeighborsClassifier(neighbors)
            
            ## Fit on the tt data
            knn.fit(pca_tt, y_tt)

            ## Get the holdout prediction
            pred = knn.predict(pca_ho)

            pca_accs[i,j,k] = accuracy_score(y_ho, pred)
            pca_accs2[i,j,k] = accuracy_score(y_tt, knn.predict(pca_tt))
            
            
            k = k + 1
        j = j + 1
    i = i + 1

CV Split 0
CV Split 1
CV Split 2
CV Split 3
CV Split 4


##### The above CV analysis with KNN takes ~10 min

In [78]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.unravel_index(np.argmax(np.mean(pca_accs, axis=0), axis=None), 
                                       np.mean(pca_accs, axis=0).shape)

print(max_index)
print(ks[max_index[1]])
print(comps[max_index[0]])
print(np.round(comps[max_index[0]],2))
print(pca_accs.shape)
print("The pair with the highest AVG CV Accuracy was",
         "k =", ks[max_index[1]],
         "and number of components =", np.round(comps[max_index[0]],2))
print("The highest AVG CV validation Accuracy was", np.max(np.mean(pca_accs, axis=0)))

print("The AVG CV training Accuracy for this pair was", np.mean(pca_accs2[:,max_index[0], max_index[1]]))


(17, 0)
1
19
19
(5, 18, 10)
The pair with the highest AVG CV Accuracy was k = 1 and number of components = 19
The highest AVG CV validation Accuracy was 0.9378278794147888
The AVG CV training Accuracy for this pair was 1.0


##### It looks like the ideal PCA dimension to reduce to was the highest we tested still

#### Let's try out some small NN's:

In [80]:
from sklearn.neural_network import MLPClassifier

In [81]:
mlp = MLPClassifier(hidden_layer_sizes=(5,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.9011689691817216

In [82]:
mlp = MLPClassifier(hidden_layer_sizes=(8,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.8937300743889479

In [83]:
mlp = MLPClassifier(hidden_layer_sizes=(8,8,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.9075451647183846

In [84]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50,5,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.9399574920297555

In [86]:
mlp = MLPClassifier(hidden_layer_sizes=(1024,128,128,24,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

/home/birdsong/.local/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


0.9516471838469713

In [105]:
mlp = MLPClassifier(hidden_layer_sizes=(1024,128,128,128,128,24,),
                    max_iter=10000)

mlp.fit(X_train, y_train)

accuracy_score(mlp.predict(X_test), y_test)

0.963336875664187

In [108]:
accuracy_score(mlp.predict(X_train), y_train)

1.0

##### Something must be weird with the MLPClassifier if the accuracy scores are all the same.

#### Random forests with CV:

In [106]:
from sklearn.ensemble import RandomForestClassifier

In [116]:
## note this will take a minute or so to run

max_depths = range(1, 11)
n_trees = [100,500]

## Make an array of zeros that will hold the cv accuracies
rf_accs = np.zeros((5, len(max_depths), len(n_trees)))
rf_accs2 = np.zeros((5, len(max_depths), len(n_trees)))


i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    ## Loop through the max_depth options
    j = 0
    for depth in max_depths:
        ## Look through the number of estimators options
        k = 0
        for n_est in n_trees:
            ## This will help you keep track of where the loop is
            print(i,j,k)
            ## Make the model object, include a random state
            rf = RandomForestClassifier(max_depth=depth,
                                        n_estimators=n_est,
                                        max_samples = int(.8*len(tt)))
            
            ## Fit the model
            rf.fit(tt, y_tt)
            
            ## predict on the holdout set
            pred = rf.predict(ho)
            
            ## Record the accuracy
            rf_accs[i,j,k] = accuracy_score(y_ho,  pred)
            #rf_accs2[i,j,k] = accuracy_score(y_tt, rf.predict(pca_tt))
            k = k + 1
        j = j + 1
    i = i + 1

0 0 0
0 0 1
0 1 0
0 1 1
0 2 0
0 2 1
0 3 0
0 3 1
0 4 0
0 4 1
0 5 0
0 5 1
0 6 0
0 6 1
0 7 0
0 7 1
0 8 0
0 8 1
0 9 0
0 9 1
1 0 0
1 0 1
1 1 0
1 1 1
1 2 0
1 2 1
1 3 0
1 3 1
1 4 0
1 4 1
1 5 0
1 5 1
1 6 0
1 6 1
1 7 0
1 7 1
1 8 0
1 8 1
1 9 0
1 9 1
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
2 3 0
2 3 1
2 4 0
2 4 1
2 5 0
2 5 1
2 6 0
2 6 1
2 7 0
2 7 1
2 8 0
2 8 1
2 9 0
2 9 1
3 0 0
3 0 1
3 1 0
3 1 1
3 2 0
3 2 1
3 3 0
3 3 1
3 4 0
3 4 1
3 5 0
3 5 1
3 6 0
3 6 1
3 7 0
3 7 1
3 8 0
3 8 1
3 9 0
3 9 1
4 0 0
4 0 1
4 1 0
4 1 1
4 2 0
4 2 1
4 3 0
4 3 1
4 4 0
4 4 1
4 5 0
4 5 1
4 6 0
4 6 1
4 7 0
4 7 1
4 8 0
4 8 1
4 9 0
4 9 1


In [118]:
## This gives you the optimal depth and number of trees
max_index = np.unravel_index(np.argmax(np.mean(rf_accs, axis=0), axis=None), 
                                       np.mean(rf_accs, axis=0).shape)


print(max_depths[max_index[0]],n_trees[max_index[1]])

10 500


In [121]:
## What is the best avg cv accuracy? (Note here we only tried it out for 100 classifiers, hence the index of 0 below)
np.mean(rf_accs, axis = 0)[max_depths[max_index[0]]-1, 0]

0.8618710797624043

In [122]:
np.mean(rf_accs, axis = 0)[max_depths[max_index[0]]-1, 1]

0.8656220616084772

#### Bayes Classifiers:

In [109]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [110]:
## Make a holder for the accuracies
bayes_accs = np.zeros((5,3))


i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    ## Linear Discriminant Analysis
    lda = LinearDiscriminantAnalysis()
    
    lda.fit(tt, y_tt)
    lda_pred = lda.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 0] = accuracy_score(y_ho,
                                      lda_pred)
    
    ## Quadratic Discriminant Analysis
    qda = QuadraticDiscriminantAnalysis()
    
    qda.fit(tt,y_tt)
    
    qda_pred = qda.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 1] = accuracy_score(y_ho,
                                      qda_pred)
    
    
    ## Gaussian Naive Bayes
    nb = GaussianNB()
    
    nb.fit(tt, y_tt)
    nb_pred = nb.predict(ho)
    
    ## Records the accuracies
    bayes_accs[i, 2] = accuracy_score(y_ho,
                                      nb_pred)
    
    i = i + 1

/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/birdsong/.local/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [111]:
## Print the Average CV Accuracies here
np.mean(bayes_accs, axis=0)

array([0.93266994, 0.60099385, 0.88306449])

In [112]:
bayes_accs

array([[0.9254571 , 0.60056259, 0.87435537],
       [0.9456165 , 0.60759494, 0.89404594],
       [0.93014534, 0.5827473 , 0.87763713],
       [0.93717768, 0.61462729, 0.88654477],
       [0.9249531 , 0.59943715, 0.88273921]])

#### Log Reg with PCA first:

In [113]:
import time

In [114]:
st = time.time()
dimst = time.time()

comps = range(2,30)
n_splits = 5
## Makes a 3-D array of to record the accuracies in
log_reg_pca_accs = np.zeros((n_splits, len(comps)))
log_reg_pca_accs2 = np.zeros((n_splits, len(comps)))

i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    ## Keeps track of the cross validation split you are on
    ## This loop can be a little long
    print("CV Split", i)
    tt = X_train[train_index]
    ho = X_train[test_index]
    y_tt = y_train[train_index]
    y_ho = y_train[test_index]
    
    j = 0
    for n_comps in comps:
        dimst = time.time()
        ## Make the PCA pipeline here
        pca_pipe = Pipeline([('scale', StandardScaler()),
                            ('pca', PCA(n_comps))])
        
        ## Fit and then get the PCA transformed tt data here
        pca_tt = pca_pipe.fit_transform(tt)
        
        ## Get the transformed holdout data here
        pca_ho = pca_pipe.transform(ho)
        
        #Logistic Regression here:
        log_reg = LogisticRegression(max_iter=10000,
                                     penalty=None)
        
        log_reg.fit(tt, y_tt)
        
        pred = log_reg.predict((ho))
        
        log_reg_pca_accs[i,j] = accuracy_score(y_ho, pred)
        log_reg_pca_accs2[i,j] = accuracy_score(y_tt, log_reg.predict(tt))
        
        j = j + 1
        dimtime = time.time() - dimst
        print("num comps = %s" % n_comps)
        print(dimtime)
    i = i + 1
et = time.time() - st
print("Total time: %s" % et)

CV Split 0
num comps = 2
1.8617753982543945
num comps = 3
2.0311219692230225
num comps = 4
1.9511528015136719
num comps = 5
2.0948646068573
num comps = 6
2.1181423664093018
num comps = 7
2.237854242324829
num comps = 8
2.1872377395629883
num comps = 9
2.0633089542388916
num comps = 10
2.345895290374756
num comps = 11
2.3353073596954346
num comps = 12
2.016392230987549
num comps = 13
1.959672451019287
num comps = 14
1.9969921112060547
num comps = 15
2.2473320960998535
num comps = 16
2.2394721508026123
num comps = 17
2.053900957107544
num comps = 18
2.774163246154785
num comps = 19
2.4493117332458496
num comps = 20
2.2285797595977783
num comps = 21
2.643915891647339
num comps = 22
2.3413989543914795
num comps = 23
2.080789804458618
num comps = 24
2.018993377685547
num comps = 25
2.1512701511383057
num comps = 26
2.152663469314575
num comps = 27
2.0492873191833496
num comps = 28
2.0650150775909424
num comps = 29
2.378972053527832
CV Split 1
num comps = 2
1.9828529357910156
num comps = 3
1

In [117]:
## This code will print out the best # components - k combo for you
## It also prints out the highest AVG CV Accuracy
max_index = np.argmax(np.mean(log_reg_pca_accs, axis=0))

print(max_index)

print("The case with the highest AVG CV Accuracy had number of components =", np.round(comps[max_index],2))
print("The highest AVG CV Accuracy was", np.max(np.mean(log_reg_pca_accs, axis=0)))

print("The corresponding AVG CV Accuracy was", np.mean(log_reg_pca_accs2[:,max_index]))

0
The case with the highest AVG CV Accuracy had number of components = 2
The highest AVG CV Accuracy was 0.940641170773928
The corresponding AVG CV Accuracy was 1.0
